In [3]:
# step 1: import dem libraries kiddos!
import json
import pandas as pd
import numpy as np
import nltk as nltk
import spacy
import os
import re 
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import *
porter_stemmer = PorterStemmer()
import nltk.data
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import TreebankWordTokenizer
treebank_tokenizer = TreebankWordTokenizer()
from nltk.util import bigrams 



In [4]:
# import the diaglogue Json Files!!!!!

wd = os.getcwd() 
# grab the locations of those lil jsons
json_files = [pos_json for pos_json in os.listdir(wd + '\\diag_jsons\\') if pos_json.endswith('.json')]

# read those guys in!

# the first step will be to READ each file, then the NEXT step will be to store it in something that will EVENTUALLY 
# be converted into a dataframe (actually, series)

# so I reckon the way to start this bad boy off is to just store them all in a SHINY NEW DICTIONARY

# and let the key be the movie name  


In [24]:
# I commented out an example. 
#with open(wd+ '\\diag_jsons\\'+json_files[0], "r") as read_file:
#    data = json.load(read_file)
#json_files[0].replace('.json','')

In [5]:
# found this on stack overflow
def remove_multiple_strings(cur_string, replace_list,low= False):
    for cur_word in replace_list:
        if low == False:
            cur_string = cur_string.replace(cur_word, '')
        else:
            cur_string = cur_string.replace(cur_word.lower(), '')
    return cur_string

In [34]:
# create a new dictionary where each key will be the movie and each value will contain all of the 
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(lemmatized_words) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [29]:
#char_series.head()

mytest = char_series[0]
sentTest = sentence_tokenizer.tokenize(mytest)
sentences_words = [treebank_tokenizer.tokenize(sentence) for sentence in sentTest]
all_tokens = [word for sentence in sentences_words for word in sentence]
bigrams = nltk.bigrams(all_tokens)
new_text = ''
bgs = ["***".join(gram) for gram in bigrams ]
bigramz = " ".join(bgs)
bigramz
#for i in bigrams:
#    gram = "***".join(i)
#    print(gram)



str

In [15]:
#miniboi = pd.Series()
#for i in mov_dict['10-Things-I-Hate-About-You_script']['dialogues'].keys():
#    miniboi.at[i+'____''10-Things-I-Hate-About-You_script'] = mov_dict['10-Things-I-Hate-About-You_script']['dialogues'][i]
#miniboi



# ok, so I suppose we will need to figure out how we will create the new index for the Pandas.Series
# heck, I suppose I could even add a movie title AFTER... but meh, I'm lazy... I think I'll just leave it as the series for now! 


TypeError: 'generator' object is not subscriptable

In [126]:
#thingz = sentence_tokenizer.tokenize(miniboi[0])
# so... we need to do this for every sentence.... YAY.... 
#lemz = ''
# I think I need to get rid of punctuation unless lemmatization gets rid of it 
#for sentence in thingz:
#    toks = treebank_tokenizer.tokenize(sentence)
#    lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
#    lemz += " ".join(lemmatized_words) + ' '


In [30]:
#lemz

In [31]:
# for examples sake... run from the line that says mniboi  until the one that says miniboi[0]
#miniboi

#miniboi[0] = re.sub("\\n\[\d+\]",'',miniboi[0])
#         char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])

#miniboi[0]

# you want to lemmatize the the text or tokenize etc, 

# tokenize by sentence 1st...then apply the lemmatization to each sentence

# then we have to find a way to concatenate or .join everything together across all of the lists!!! 
#miniboi[0]
# Do some Data preprocessing on the text
# maybe tokenizing, stemming or lemmatizing will get rid of the bad woyds!
# so now I know to do this to the text to preprocess it, again, I can do it in the initial loop, or after. 
# it's kinda whatever. 

# time to lemmatize!!!!
# or friggin tokenize, at least!
#thingz = sentence_tokenizer.tokenize(miniboi[0])
#print(thingz)
#lemz = []
#for sentence in thingz:
#    lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in sentence]
#    lemz += lematized_words
#lemmatized_words
#" ".join(["John", "Charles", "Smith"])

# Time to get those term counts!!!!!
#lemz
#miniboi[0]

In [14]:
# LSA, LSA!!!!
# wondeirng what the count_vecotrizer does??? Here's the basic documentation: 
# Convert a collection of text documents to a matrix of token counts
#This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.
#If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature selection then the number of features will be equal to the vocabulary size found by analyzing the data.
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
#display(document_term_matrix)

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [33]:
# Let's reduce the dimensionality of that Document term matrix and do some clustering! 

# I"m doing this for 2 reasons: 1, I essentially copy pasted from Stephen 

#2:.... we have a lot of data.... let's make the run time as fast as possible!


In [32]:
# Define helper functions, These are from Stephen
def get_keys(topic_matrix):
    '''returns an integer list of predicted topic categories for a given topic matrix'''
    keys = []
    for i in range(topic_matrix.shape[0]):
        keys.append(topic_matrix[i].argmax())
    return keys
###################################################################################
def keys_to_counts(keys):
    '''returns a tuple of topic categories and their accompanying magnitudes for a given list of keys'''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

##############################################################################
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''returns a list of n_topic strings, where each string contains the n most common 
        words in a predicted category, in order'''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [36]:
count_vectorizer = CountVectorizer(ngram_range = (4,4))
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)
for i in range(5,31):
    n_topics = i

    lsa_model = TruncatedSVD(n_components=n_topics)
    lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

    lsa_keys = get_keys(lsa_topic_matrix)
    lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

    top_n_words_lsa = get_top_n_words(5, lsa_keys, document_term_matrix, count_vectorizer)

    for i in range(len(top_n_words_lsa)):
        print("Topic {}: ".format(i), top_n_words_lsa[i])
    print('********')

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11075x4856542 sparse matrix of type '<class 'numpy.int64'>'
	with 5480525 stored elements in Compressed Sparse Row format>

Topic 0:  deblois sanders final draft draft 02 13 2010 sanders final draft 02 dragon deblois sanders final final draft 02 13
Topic 1:  what are you doing you re going to you re gon na we re going to what do you mean
Topic 2:  pink revision 20 07 pink revision 19 07 yellow revision 20 07 real slow real slow one of the most
Topic 3:  am am am am approaching newark station newark to camera to camera now approaching newark station station newark station next
Topic 4:  la la la la the scene dissolve to the scene cut to scene cut to the in front of him
********
Topic 0:  sanders final draft 02 deblois sanders final draft draft 02 13 2010 dragon deblois sanders final final draft 02 13
Topic 1:  what are you doing you re going to you re gon na what do you mean we re going to
Topic 2:  red rum red rum rum red rum red you may be sure in the midst of in the world the
Topic 3:  to camera to camera camera to camera to in and out of in the history of the heart of the
Topic 4:  pink revision 20 07 fr

Topic 6:  green script of 09 valentine official green script official green script of blue valentine official green japanese no subtitle japanese
Topic 7:  get out of here you re gon na took care of the wa at that dinner out of here you
Topic 8:  from original script sequence sequence from original script script sequence from original original script sequence from sequence ted from original
Topic 9:  thank you thank you to camera to camera camera to camera to real slow real slow thank you so much
Topic 10:  you ll have to the edge of the we ll have to what you re saying nellis air force base
Topic 11:  pink revision 20 07 cedar rapid 12 09 red rum red rum rum red rum red pink revision 19 07
Topic 12:  blue revision 03 26 revision 03 26 10 4th blue revision 03 and the warrior three what do you mean
********
Topic 0:  draft 02 13 2010 final draft 02 13 sanders final draft 02 dragon deblois sanders final deblois sanders final draft
Topic 1:  what are you doing you re going to you re gon n

Topic 7:  blue valentine official green green script of 09 valentine official green script official green script of no subtitle japanese no
Topic 8:  yes mr yes mr seen evil dead ii nothing to do with mr yes mr yes what do you mean
Topic 9:  thank you thank you thank you so much do you hear the my name is cohen you do have to
Topic 10:  we ll have to you ll have to the edge of the he shake his head she shake her head
Topic 11:  you re looking for we re going to what we re doing to be able to oh dear oh dear
Topic 12:  4th blue revision 03 blue revision 03 26 revision 03 26 10 and the warrior three the side of the
Topic 13:  am am am am camera to camera to you re gon na to camera to camera what do you mean
Topic 14:  from original script sequence sequence from original script script sequence from original original script sequence from ted from original script
Topic 15:  cedar rapid 12 09 red rum red rum rum red rum red will overload in minute engine will overload in
Topic 16:  you re go

Topic 6:  get out of here out of here you you re gon na took care of the wa at that dinner
Topic 7:  valentine official green script green script of 09 blue valentine official green official green script of subtitle japanese no subtitle
Topic 8:  yes mr yes mr seen evil dead ii have seen evil dead what do you mean evil dead ii yet
Topic 9:  thank you thank you thank you so much you do have to do you hear the my name is cohen
Topic 10:  the edge of the we ll have to you ll have to the back of the he shake his head
Topic 11:  blue revision 03 26 revision 03 26 10 4th blue revision 03 you re looking for and the warrior three
Topic 12:  it thrill to be keep an eye on you re supposed to did you see the what do you mean
Topic 13:  ha ha ha ha the sound of the the lequint dickey mining lequint dickey mining company when it come to
Topic 14:  my name is and where are you from name is and want want to recruit you is and want to
Topic 15:  head of mall security out of the way to be the one you t

Topic 15:  we ve got to pick up the phone we just have to there only one way ve got to think
Topic 16:  what do you mean it thrill to be keep an eye on her against the wall to keep an eye
Topic 17:  pink revision 20 07 pink revision 19 07 head of mall security to camera to camera yellow revision 20 07
Topic 18:  you know what mean none of my business it none of my get the fuck out the fuck out of
Topic 19:  we re gon na you re gon na what do you think you want me to what do you mean
Topic 20:  cedar rapid 12 09 from original script sequence sequence from original script original script sequence from script sequence from original
Topic 21:  red rum red rum rum red rum red and then you re then you re gone you re going to
Topic 22:  revision 03 26 10 blue revision 03 26 4th blue revision 03 and the warrior three what are you doing
Topic 23:  what do you mean what the matter with you re going to we re going to the side of the
Topic 24:  no no no no that ai no crime no crime that ai crime t

Topic 16:  you want me to it ha to be we re going to looked at it cause got ta do first
Topic 17:  take look at me st john the divine look at me now there he said now you familiar with the
Topic 18:  head of mall security you re gon na it none of my get out of here none of my business
Topic 19:  am am am am you know what mean will overload in minute engine will overload in you re you re
Topic 20:  pink revision 20 07 pink revision 19 07 no no no no camera to camera to to camera to camera
Topic 21:  no no no no what are you doing what are you talking arctic warrior arctic warrior are you talking about
Topic 22:  blue revision 03 26 4th blue revision 03 revision 03 26 10 and the warrior three what are you doing
Topic 23:  we re gon na united state of america the united state of it gon na be come on baby light
Topic 24:  from original script sequence sequence from original script script sequence from original original script sequence from sequence ted from original
Topic 25:  that ai no c

In [36]:
# Now we will do LSA with tokenization instead of lemmatization, just so that the outputted words make more sense!
# Hopefully there's a way to do this with sequences of 2-3 words, because some of these are gibberish.


In [22]:
# create a new dictionary where each key will be the movie and each value will contain all of the 
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            #lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(toks) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [26]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)

n_topics = i

lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

top_n_words_lsa = get_top_n_words(20, lsa_keys, document_term_matrix, count_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i), top_n_words_lsa[i])


C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11075x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 2789943 stored elements in Compressed Sparse Row format>

Topic 0:  youhhh ihhh hhh ithhh hhhi dohhhn thathhh mehhh hhhyou ll hhhand ihhhdo whathhh wehhh ve nohhh inhhhthe herehhh hehhh therehhh
Topic 1:  hhh hhhthe hhhand hhhi uhhh ofhhhthe inhhhthe youhhh ithhh hhhyou hhhwe tohhhthe herehhh onhhhthe hhha mr hhhu hhhhe ihhh thathhh
Topic 2:  ofhhhthe inhhhthe tohhhthe onhhhthe hhhand hhhthe athhhthe ithhh himhhh intohhhthe fromhhhthe thehhhdoor hhh andhhhthe hhha outhhhof hhhhe uphhh hehhh doorhhh
Topic 3:  hhhand hhhi ofhhhthe hhhthe inhhhthe hhhbut mehhh ihhhwas tohhhthe ithhhwas ihhham hhh andhhhthe ihhhhad hhhhe tohhhbe ithhh hhhas thathhhi forhhhthe
Topic 4:  yeahhhh ithhh thathhh heyhhh righthhh whathhh uphhh lethhh hehhh dohhhn ohhhh gohhh wehhh onhhh shhhgo hhhwe shhhthe gonhhhna hhhyeah hhhwhat
Topic 5:  youhhh hhhyou rumhhh youhhhare hhhand youhhhknow thathhh wellhhh manhhh ofhhhthe tohhhyou timeshhh athhhall thishhh ithhhis dohhhn youhhhhave whathhh youhhhready thankhhhyou
Topic 6:  aihhhn thhhno thathhhai crimehhh ihhh gonhhhna m

In [ ]:
# THERE IS A graph algo called infomap... code available...all you provide it is a graph????? 

# i.e. character1 movie1 compared to character 7 movie 3... put the similarity in there.... 

# benefit of info map is it will just see it as 1 big grph and builds communities inside of graph... 


# pici villain from jaems bond movie... pick departed.. if you find bond character 1 and 

# we can remove stopwords... before 

# we can go oline and downlaoad it.... 

# mapequation.org.. it's a C++ implementation..... so we put in the graph..... 

In [29]:
# create a new dictionary where each key will be the movie and each value will contain all of the 
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            bigrams = nltk.bigrams(toks)
            bgs = ["_".join(gram) for gram in bigrams ]
                
                    
            #lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(bgs) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [31]:
char_series[0]

"leave_it it_i i_said said_, ,_leave leave_it it_! why_did did_n't n't_we we_just just_read read_the the_hardy hardy_boys boys_? this_book book_is is_about about_a a_guy guy_and and_his his_fishing fishing_habit habit_. not_exactly exactly_a a_crucial crucial_topic topic_. frankly_, ,_i i_'m 'm_baffled baffled_as as_to to_why why_we we_still still_revere revere_hemingway hemingway_. he_was was_an an_abusive abusive_, ,_alcoholic alcoholic_misogynist misogynist_who who_had had_a a_lot lot_of of_cats cats_. i_guess guess_the the_school school_board board_thinks thinks_because because_hemingway hemingway_'s 's_male male_and and_an an_asshole asshole_, ,_he he_'s 's_worthy worthy_of of_our our_time time_what what_about about_colette colette_? charlotte_bronte bronte_? simone_de de_beauvoir beauvoir_? expressing_my my_opinion opinion_is is_not not_a a_terrorist terrorist_action action_. i_still still_maintain maintain_that that_he he_kicked kicked_himself himself_in in_the the_balls balls_.

In [19]:
text_sample

array(["leave***it it***i i***said said***, ,***leave leave***it it***! why***did did***n't n't***we we***just just***read read***the the***hardy hardy***boys boys***? this***book book***is is***about about***a a***guy guy***and and***his his***fishing fishing***habit habit***. not***exactly exactly***a a***crucial crucial***topic topic***. frankly***, ,***i i***'m 'm***baffled baffled***as as***to to***why why***we we***still still***revere revere***hemingway hemingway***. he***was was***an an***abusive abusive***, ,***alcoholic alcoholic***misogynist misogynist***who who***had had***a a***lot lot***of of***cats cats***. i***guess guess***the the***school school***board board***thinks thinks***because because***hemingway hemingway***'s 's***male male***and and***an an***asshole asshole***, ,***he he***'s 's***worthy worthy***of of***our our***time time***what what***about about***colette colette***? charlotte***bronte bronte***? simone***de de***beauvoir beauvoir***? expressing***my m

In [20]:
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)

<11075x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1406463 stored elements in Compressed Sparse Row format>

  (0, 591)	2
  (0, 646)	2
  (0, 3637)	2
  (0, 3660)	2
  (0, 3977)	2
  (0, 6030)	2
  (0, 1300)	2
  (0, 1290)	2
  (0, 7739)	2
  (0, 4034)	2
  (0, 1326)	2
  (0, 1118)	2
  (0, 971)	2
  (0, 1771)	2
  (0, 9910)	2
  (0, 5037)	2
  (0, 7388)	2
  (0, 8015)	2
  (0, 1096)	2
  (0, 1859)	2
  (0, 2874)	2
  (0, 8475)	2
  (0, 2833)	2
  (0, 4060)	2
  (0, 2308)	2
  :	:
  (11074, 5795)	2
  (11074, 5935)	2
  (11074, 6110)	2
  (11074, 3324)	2
  (11074, 9101)	2
  (11074, 1866)	2
  (11074, 9822)	2
  (11074, 5090)	2
  (11074, 8086)	2
  (11074, 3245)	2
  (11074, 4935)	2
  (11074, 9948)	1
  (11074, 9910)	2
  (11074, 931)	2
  (11074, 5206)	1
  (11074, 1409)	2
  (11074, 5224)	4
  (11074, 4282)	4
  (11074, 6083)	2
  (11074, 4970)	4
  (11074, 3904)	7
  (11074, 9961)	2
  (11074, 5215)	2
  (11074, 9096)	2
  (11074, 4871)	2


In [33]:
count_vectorizer = CountVectorizer()
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)
for i in range(5,31):
    n_topics = i

    lsa_model = TruncatedSVD(n_components=n_topics)
    lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

    lsa_keys = get_keys(lsa_topic_matrix)
    lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

    top_n_words_lsa = get_top_n_words(20, lsa_keys, document_term_matrix, count_vectorizer)

    for i in range(len(top_n_words_lsa)):
        print("Topic {}: ".format(i), top_n_words_lsa[i])
    print('********')

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11075x844605 sparse matrix of type '<class 'numpy.int64'>'
	with 5112881 stored elements in Compressed Sparse Row format>

Topic 0:  you_ i_ it_ _i re do_n that_ me_ _you ll _and i_do what_ we_ ve no_ in_the here_ he_ there_
Topic 1:  of_the in_the _the to_the _and on_the at_the him_ it_ into_the from_the _a _he and_the out_of the_door up_ he_ her_ them_
Topic 2:  _and _i of_the _the in_the _6 revision_ 07 _but 20 me_ to_the i_am i_was and_the pink_revision it_ it_was yes_ _a
Topic 3:  yeah_ rum_ red_rum _you hey_ ai_n right_ it_ that_ai let_ t_no no_crime that_ crime_ what_ you_know mack_ again_ on_ here_
Topic 4:  you_ _you re mr you_are times_ all_times gentlemen_ at_all sir_ ready_ this_ order_ here_ you_ready thank_you mr_vaughn re_a _let please_
********
Topic 0:  you_ i_ it_ _i re do_n that_ me_ _you ll _and i_do what_ we_ ve no_ in_the here_ he_ there_
Topic 1:  of_the in_the _the to_the _and on_the at_the him_ it_ into_the _a from_the _he and_the out_of the_door up_ he_ her_ them_
Topic 2:  _and _i of_the _the in_the _6 07 revision_ _but 20 me_ to_the i_am i_was pink_revision and_the it_ it_was _a

KeyboardInterrupt: 